- https://nijianmo.github.io/amazon/index.html
- https://jmcauley.ucsd.edu/data/amazon/ 
- https://colab.research.google.com/drive/1Zv6MARGQcrBbLHyjPVVMZVnRWsRnVMpV

In [3]:
prfx = 'prep_20210302A1'

# get data

    download from http://localhost:8080/notebooks/git/product-category/notebooks/secrets_get_amazon_data.ipynb

In [4]:
import pandas as pd
import gzip
import json

In [5]:
pdata = "/data/git/product-category/data/amazon_meta_data"
!ls -hl {pdata}
fnms = !ls {pdata}
fnms = [o for o in fnms if o.endswith('.json.gz')]
domains = [o.strip('meta_').strip('.json.gz') for o in fnms]
dmn2fnm = dict(zip(domains, fnms))
dmn2fnm

total 13G
-rw-rw-r-- 1 ubuntu ubuntu   30M Dec  2 00:07 meta_AMAZON_FASHION.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  9.6M Aug  7  2020 meta_All_Beauty.json.gz
-rw-rw-r-- 1 ubuntu ubuntu   57M Aug  7  2020 meta_Appliances.json.gz
-rw-rw-r-- 1 ubuntu ubuntu   57M Aug  7  2020 meta_Appliances.json.gz.1
-rw-rw-r-- 1 ubuntu ubuntu  125M Aug  7  2020 meta_Arts_Crafts_and_Sewing.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  1.5G Aug  7  2020 meta_Automotive.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  1.2G Aug  7  2020 meta_Books.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  1.2G Aug  7  2020 meta_Books.json.gz.1
-rw-rw-r-- 1 ubuntu ubuntu  153M Aug  7  2020 meta_CDs_and_Vinyl.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  333M Aug  7  2020 meta_Cell_Phones_and_Accessories.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  1.5G Dec  2 00:18 meta_Clothing_Shoes_and_Jewelry.json.gz
-rw-rw-r-- 1 ubuntu ubuntu   12M Aug  7  2020 meta_Digital_Music.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  1.2G Aug  7  2020 meta_Electronics.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  37

{'AMAZON_FASHION': 'meta_AMAZON_FASHION.json.gz',
 'All_Beauty': 'meta_All_Beauty.json.gz',
 'Appliance': 'meta_Appliances.json.gz',
 'Arts_Crafts_and_Sewi': 'meta_Arts_Crafts_and_Sewing.json.gz',
 'Automotive': 'meta_Automotive.json.gz',
 'Book': 'meta_Books.json.gz',
 'CDs_and_Vinyl': 'meta_CDs_and_Vinyl.json.gz',
 'Cell_Phones_and_Accessorie': 'meta_Cell_Phones_and_Accessories.json.gz',
 'Clothing_Shoes_and_Jewelry': 'meta_Clothing_Shoes_and_Jewelry.json.gz',
 'Digital_Music': 'meta_Digital_Music.json.gz',
 'Electronic': 'meta_Electronics.json.gz',
 'Gift_Card': 'meta_Gift_Cards.json.gz',
 'Grocery_and_Gourmet_Food': 'meta_Grocery_and_Gourmet_Food.json.gz',
 'Home_and_Kitche': 'meta_Home_and_Kitchen.json.gz',
 'Industrial_and_Scientific': 'meta_Industrial_and_Scientific.json.gz',
 'Kindle_Store': 'meta_Kindle_Store.json.gz',
 'Luxury_Beauty': 'meta_Luxury_Beauty.json.gz',
 'Magazine_Subscripti': 'meta_Magazine_Subscriptions.json.gz',
 'Movies_and_TV': 'meta_Movies_and_TV.json.gz',
 

In [4]:
KEYS2USE = set(['category', 'description', 'title', 'brand', 'feature', 'asin'])

In [5]:
# https://colab.research.google.com/drive/1Zv6MARGQcrBbLHyjPVVMZVnRWsRnVMpV#scrollTo=7igYuRaV4bF7
    
### load the meta data
def get_meta_data(domain, nrows=None):
    keys2use = KEYS2USE
    fnm = dmn2fnm[domain]
    data = []
    with gzip.open(f'{pdata}/{fnm}') as f:
        for i,l in enumerate(f):
            dat = json.loads(l.strip())
            dat = {k:v for k,v in dat.items() if k in keys2use}
            data.append(dat)
            if nrows and i>nrows: break

    # total length of list, this number equals total number of products
#     print("len(data)", len(data))

    # convert list into pandas dataframe
    df = pd.DataFrame.from_dict(data)
#     print("len(df)", len(df))

    ### remove rows with unformatted title (i.e. some 'title' may still contain html style content)
    df3 = df.fillna('')
    df4 = df3[df3.title.str.contains('getTime')] # unformatted rows
    df5 = df3[~df3.title.str.contains('getTime')] # filter those unformatted rows
#     print("len(df4)", len(df4))
#     print("len(df5)", len(df5))

    return df5

In [6]:
df = get_meta_data(domains[3], nrows=10000)
df.head(3)

,category,description,title,brand,feature,asin
0,"[Arts, Crafts & Sewing, Sewing, Trim & Embelli...",[The patch features the sweaty masculine hands...,You Son of a Bitch! 1987 Embroidered Patch,Honchosfx,[You son of a bitch patch - exclusive to Honch...,6665560953
1,"[Arts, Crafts & Sewing, Crafting, Paper & Pape...","[With 5 packs stars folding paper, each pack a...",Origami Stars Papers Package 1H (5 packs),,[],7000000376
2,"[Arts, Crafts & Sewing, Painting, Drawing & Ar...",[],Yi De Ge Chinese Calligraphy Sumi Drawing Blac...,MasterChinese,[],7000001089


# preprocess data

In [11]:
dfs = []
for dmn in domains:
    df = get_meta_data(dmn, nrows=None)
    if set(df.columns)!= KEYS2USE: continue
    df = df[df.category.apply(len)>0]
    if len(df)==0: continue
    print(dmn, df.shape)
    df['domain'] = dmn
    display(df.sample(2))
    dfs.append(df)

Appliance (29639, 6)


,category,description,title,brand,feature,asin,domain
9139,"[Appliances, Laundry Appliances, Washers & Dry...",[Samsung WF457ARGSWR 4.5 cu. ft. King-size Cap...,Samsung WF457ARGSWR 4.5 Cu. Ft. White Stackabl...,Samsung,[Control and Monitor the Washer from your Smar...,B008493D4Y,Appliance
27786,"[Appliances, Parts & Accessories]",[GE Part Number WE4M190 TIMER this is an O.E.M...,GE WR2X9358 General Electric Cover Snack Dish SM,GE,"[This is an O.E.M. authorized part, The O.E.M ...",B018LG9IUC,Appliance


Arts_Crafts_and_Sewi (284889, 6)


,category,description,title,brand,feature,asin,domain
270108,"[Arts, Crafts & Sewing, Crafting, Craft Suppli...","[* S A L E * Buy any 2 Embolden Jewelry item, ...",Charm Do It Yourself [DIY] Purple Glitters Ros...,Embolden Jewelry,[],B018X4J2GC,Arts_Crafts_and_Sewi
106477,"[Arts, Crafts & Sewing, Scrapbooking & Stampin...",[Cling Rubber is made of the same deeply etche...,"STAMPENDOUS CRM291 Cling Rubber Stamp, Literar...",STAMPENDOUS,[Rubber guarantees durability and definition o...,B00AYRO2BM,Arts_Crafts_and_Sewi


Automotive (897587, 6)


...

,category,description,title,brand,feature,asin,domain
136726,"[Tools & Home Improvement, Lighting & Ceiling ...","[Solar Power, Outdoor Solar Garden Lights, Sta...","Solar Power, Outdoor Solar Landscape Path Ligh...",Sana Enterprises,"[Pack of 12 solar path landscape lights, Each ...",B002KML8UW,Tools_and_Home_Improvement
27259,"[Tools & Home Improvement, Electrical, Cords, ...","[<div class=""aplus"">, The Tower Manufacturing ...","Tower Manufacturing 30338018 18&quot; Length, ...",Tower,"[18-inch, 15 amp in-line ground fault circuit ...",B0007NOZLA,Tools_and_Home_Improvement


Toys_and_Game (571640, 6)


,category,description,title,brand,feature,asin,domain
70215,"[Toys & Games, Hobbies, Trains & Accessories, ...",[Features: * Parts for Six Complete Tanks: 2 H...,Walthers Cornerstone N Scale Modulars Storage ...,Walthers,"[Mix and match walls, doors, windows and detai...",B000UCLJXK,Toys_and_Game
138219,"[Toys & Games, Kids' Electronics, Electronic P...",[Tons of fun for you and your favorite Zhu Zhu...,Zhu Zhu Pets Adventure Ball,Zhu Zhu Pets,"[Zhu Zhu Pet Hamster Deluxe Accessories, Inclu...",B002DCS3R0,Toys_and_Game


Video_Game (82394, 6)


,category,description,title,brand,feature,asin,domain
59892,"[Video Games, PC, Games]",[legends of terror 6 pack bonus edition pc-dvd...,legends of terror 6 pack bonus edition pc-dvd-...,Viva Media,[Ships fast.],B00DB6M77O,Video_Game
39902,"[Video Games, Xbox 360, Games, </span></span><...",[Assume the role of Union and Confederate sold...,History Channel Civil War: Secret Missions - X...,by\n \n Activision,[Assume the role of Union and Confederate sold...,B001CDL6SY,Video_Game


In [12]:
df = pd.concat(dfs)
df.shape

(13706665, 7)

In [13]:
df.sample(3)

,category,description,title,brand,feature,asin,domain
36234,"[Sports & Outdoors, Sports & Fitness, Hunting ...",[The Betts 700-10-64 1/64-Ounce Leadhead has g...,"Betts 700-10-64 1/64-Ounce Leadhead, Black Finish",Betts,"[12 Packs, 10 pieces per pack]",B000LF38DU,Sports_and_Outdoor
860336,"[Clothing, Shoes & Jewelry, Costumes & Accesso...","[For imported goods, if there are some dents, ...",Men Mask Blank Male Mask cosplay mask white (j...,JBW,[It is a mask of type (Jabau~okki's) is using ...,B00CMHMK5M,Clothing_Shoes_and_Jewelry
2274318,"[Books, Engineering &amp; Transportation, Auto...","[, ]",BMW Motorcycles: The Complete Story (Crowood A...,Bruce Preston,[],1861260059,Book


In [14]:
df.to_csv(f'../data/data__{prfx}.csv', index=False)

In [15]:
ls -hl ../data/data__{prfx}.csv

-rw-rw-r-- 1 ubuntu ubuntu 13G Mar  2 18:06 ../data/data__prep_20210302A1.csv


## make sample

In [7]:
%%time
df_sample = pd.read_csv(f'../data/data__{prfx}.csv')
df_sample.head()

CPU times: user 1min 50s, sys: 13.6 s, total: 2min 4s
Wall time: 2min 34s


,category,description,title,brand,feature,asin,domain
0,"['Appliances', 'Refrigerators, Freezers & Ice ...",[],Tupperware Freezer Square Round Container Set ...,Tupperware,['Each 3-pc. set includes two 7/8-cup/200 mL a...,7301113188,Appliance
1,"['Appliances', 'Refrigerators, Freezers & Ice ...",['2 X Tupperware Pure & Fresh Unique Covered C...,2 X Tupperware Pure &amp; Fresh Unique Covered...,Tupperware,['2 X Tupperware Pure & Fresh Unique Covered C...,7861850250,Appliance
2,"['Appliances', 'Parts &amp; Accessories']",[],The Cigar - Moments of Pleasure,The Cigar Book,[],8792559360,Appliance
3,"['Appliances', 'Parts & Accessories']","['Multi purpost descaler, especially suited to...",Caraselle 2X 50G Appliance Descalene,Caraselle,[],9792954481,Appliance
4,"['Appliances', 'Parts & Accessories', 'Range P...",['Full gauge and size beveled-edge; furnished ...,EATON Wiring 39CH-SP-L Arrow Hart 1-Gang Chrom...,EATON Wiring,['Returns will not be honored on this closeout...,B00002N5EL,Appliance


In [12]:
import numpy as np
np.random.seed(101)
df_sample = df_sample.sample(10000)

In [13]:
df_sample.to_csv(f'../data/data_sample__{prfx}.csv', index=False)